# Kafka Demo

### Connect to Kafka Broker Server
```
ssh -o ServerAliveInterval=60 -L 9092:localhost:9092 tunnel@128.2.204.215 -NTf
```
pass: seaitunnel


To kill connection at port:
```
lsof -ti:9092 | xargs kill -9
```

### Setup
```
python -m pip install kafka-python
```

### Producer Mode -> Writes Data to Broker

In [2]:
from os import path
import sys, os
from datetime import datetime
from json import dumps, loads
from time import sleep
from random import randint
import numpy as np
# ssh -o ServerAliveInterval=60 -L 9092:localhost:9092 tunnel@128.2.24.106 -NTf
from kafka import KafkaConsumer, KafkaProducer

# Update this for your own recitation section :)
topic = 'recitation-c'

In [3]:
# Create a producer to write data to kafka
# Ref: https://kafka-python.readthedocs.io/en/master/apidoc/KafkaProducer.html

# [TODO]: Replace '...' with the address of your Kafka bootstrap server
producer = KafkaProducer(bootstrap_servers=['localhost:9092'],
                        value_serializer=lambda x: dumps(x).encode('utf-8'),
                        )

# [TODO]: Add cities of your choice
cities = ['Atlanta', 'Seattle', 'Boston']

# Write data via the producer
print("Writing to Kafka Broker")
for i in range(10):
    data = f'{datetime.now().strftime("%Y-%m-%d %H:%M:%S")},{cities[randint(0,len(cities)-1)]},{randint(18, 32)}ºC'
    print(f"Writing: {data}")
    producer.send(topic=topic, value=data)
    sleep(1)

Writing to Kafka Broker
Writing: 2024-01-25 10:33:04,Seattle,32ºC
Writing: 2024-01-25 10:33:05,Seattle,24ºC
Writing: 2024-01-25 10:33:06,Seattle,20ºC
Writing: 2024-01-25 10:33:07,Seattle,29ºC
Writing: 2024-01-25 10:33:08,Boston,18ºC
Writing: 2024-01-25 10:33:09,Seattle,18ºC
Writing: 2024-01-25 10:33:10,Seattle,32ºC
Writing: 2024-01-25 10:33:11,Boston,28ºC
Writing: 2024-01-25 10:33:12,Boston,20ºC
Writing: 2024-01-25 10:33:13,Seattle,28ºC


### Consumer Mode -> Reads Data from Broker

In [ ]:
# Create a consumer to read data from kafka
# Ref: https://kafka-python.readthedocs.io/en/master/apidoc/KafkaConsumer.html

# [TODO]: Complete the missing ... parameters/arguments using the Kafka documentation
consumer = KafkaConsumer(
    'recitation-c',
    bootstrap_servers=['localhost:9092'],
    auto_offset_reset='earliest', #Experiment with different values: earliest, latest
    # Commit that an offset has been read
    enable_auto_commit=True,
    # How often to tell Kafka, an offset has been read
    auto_commit_interval_ms=1000
)

print('Reading Kafka Broker')
for message in consumer:

    message = message.value.decode()
    # Default message.value type is bytes!
    print(loads(message))
    os.system(f"echo {message} >> kafka_log.csv")

Reading Kafka Broker
2024-01-24 21:54:22,New York City,29ºC
2024-01-24 21:54:23,Washington D.C.,21ºC
2024-01-24 21:54:24,Atlanta,25ºC
2024-01-24 21:54:25,New York City,22ºC
2024-01-24 21:54:26,Pittsburgh,31ºC
2024-01-24 21:54:27,Pittsburgh,27ºC
2024-01-24 21:54:28,Washington D.C.,31ºC
2024-01-24 21:54:29,New York City,27ºC
2024-01-24 21:54:30,Pittsburgh,25ºC
2024-01-24 21:54:31,New York City,19ºC
2024-01-25 10:33:04,Seattle,32ºC
2024-01-25 10:33:05,Seattle,24ºC
2024-01-25 10:33:06,Seattle,20ºC
2024-01-25 10:33:07,Seattle,29ºC
2024-01-25 10:33:08,Boston,18ºC
2024-01-25 10:33:09,Seattle,18ºC
2024-01-25 10:33:10,Seattle,32ºC
2024-01-25 10:33:11,Boston,28ºC
2024-01-25 10:33:12,Boston,20ºC
2024-01-25 10:33:13,Seattle,28ºC


# Use kcat!
It's a CLI (Command Line Interface). Previously known as kafkacat


Ref: https://docs.confluent.io/platform/current/app-development/kafkacat-usage.html

In [ ]:
#kcat command: connect to local Kafka broker, specify a topic, and consume messages from the earliest offset

#kcat -b localhost:9092 -t recitation-c -C -o earliest

# Kafka Topic: A topic in Kafka is a way to categorize data. Producers publish messages to specific topics, and consumers subscribe to topics to receive messages. A topic is a logical concept and can be distributed physically across multiple Kafka brokers and partitions.

# Kafka Offset: The offset in Kafka is used to uniquely identify the sequence number of each message within a topic. Each message has a unique offset within its partition. Consumers can track the last offset they have read, allowing them to continue from where they left off in the event of a disconnection or failure, thus ensuring message continuity.